In [ ]:
!pip install -r requirements.txt

In [1]:
from langame_client import LangameClient
from pprint import pprint
from firebase_admin import firestore

In [2]:
langame_client = LangameClient()

In [ ]:
# https://emojipedia.org/search
pprint(langame_client.generate_save_memes_general(topics_with_emojis = [
    ("knowledge", "🤔🧠💭📚📖"),
],
    memes_per_topic = 3,
    memes_per_meme=3,
    wikipedia_description = False,
    openai_description = False,
    anti_rate_limit_delay = 1))

In [ ]:
for t in langame_client._firestore_client.collection_group(u"tags")\
    .where(u"topic.content", u"in", ["ice breaker"])\
    .order_by(u"createdAt", direction=firestore.Query.DESCENDING)\
    .limit(10)\
    .stream():
    pprint(t.reference.parent.parent.get().to_dict())

In [ ]:
import json
memes = []
for m in langame_client._firestore_client.collection(u"memes").stream():
    v = m.to_dict()
    v['createdAt'] = v['createdAt'].timestamp()
    v['id'] = m.id
    v['tags'] = [] 
    for t in m.reference.collection('tags').stream():
        x = t.to_dict()
        x['createdAt'] = x['createdAt'].timestamp()
        x['id'] = t.id
        v['tags'].append(x)
    memes.append(v)

with open('memes.json', 'w') as outfile:
    json.dump({'memes': memes}, outfile, indent=4)


In [ ]:
import json
from datetime import datetime
with open('memes.json', 'r') as f:
    memes = json.load(f)
    pprint(memes)
    
    for m in memes.get('memes'):
        ref = langame_client._firestore_client.collection(u"memes")\
            .doc(m.get('id'))\
            .set({
                'content': m.get('content'),
                'createdAt': datetime.fromtimestamp(m.get('createdAt')),
            }).reference
        for t in memes.get('memes').get('tags'):
            ref.collection('tags').doc(t.get('id'))\
                .set({
                    'createdAt': datetime.fromtimestamp(t.get('createdAt')),
                    'topic': t.get('topic'),
                    'classification': t.get('classification'),
                    'origin': t.get('origin'),
                    'feedback': t.get('feedback'),
                    'context': t.get('context'),
                })
    


In [ ]:
for t in langame_client._firestore_client.collection_group(u"players")\
    .stream():
    
    #langame_client._firestore_client.doc(f"players")
        pprint(t.reference.delete())

In [ ]:
for t in langame_client._firestore_client.collection(u"langames")\
    .list_documents():
        pprint(t.delete())
        for p in t.collection("players").stream():
            pprint(p.reference.delete())

In [14]:
p = {}
p["type"] = "TopicGeneralist"
p["template"] = """Provide me a conversation starter that brings extremely deep conversations between people on the topic of "[TOPIC]".
Conversation starter:"""
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": "davinci-instruct-beta",
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0,
            "presencePenalty": 0,
            "stop": ["\\n"],
        }
    }
}

In [16]:
t

{'createdAt': Sentinel: Value used to set a document field to the server timestamp.,
 'engine': {'parameters': {'model': 'davinci-instruct-beta',
   'temperature': 0.7,
   'maxTokens': 150,
   'topP': 1,
   'frequencyPenalty': 0,
   'presencePenalty': 0,
   'stop': ['\\n']}}}

In [18]:
p_col = langame_client._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 7, 6, 19, 8, 14, 361479, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x7f8e491b0a60>)


In [3]:
langame_client.prompt_to_meme(topic_with_emoji = 
    ("knowledge", "🤔🧠💭📚📖"),
)

TypeError: call_completion() got an unexpected keyword argument 'topP'